<a href="https://colab.research.google.com/github/jiw3026/octopus_please_/blob/main/pytorch_kobert_pt_%EC%A0%80%EC%9E%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Module**

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import streamlit as st
import streamlit.components.v1 as stc
import joblib
import pandas as pd
import os
import altair as alt

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 14.9 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 KB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp38-cp38-linux_x86_64.whl size=689000 sha256=b752deccb584e587017109ad6ba2fcf049fb90fc018544e8599ab7bf6134026a
  Stored in directory: /root/.cache/pip/wheels/b6/93/9d/2237550c409eb3ed725d6302b7897ddd9a037b40cef66dcd9c
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-0jkelr5i
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-0jkelr5i
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 KB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.9/881.9 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook


from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
#GPU 사용
device = torch.device("cuda:0")
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
import os

# **Load Data**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
data = pd.read_csv(r'/content/drive/MyDrive/kpmg/concat.csv')

NameError: ignored

In [ ]:
data

,contents,category
0,이를 유해화학물질 취급시설을 지속적으로 개선하여 화관법 취급시설 최초 검사 결과 발...,e
1,관리 변화와 평가하기 공개합니다 따라 위험과 사용된 기후 기회를 지표를 b. Sco...,e
2,"[투입정보 및 가정, 분석방법] 국내외사업장의 18년 21년 배출량, 디스플레이 업...",e
3,향후 목표한 바를 향해 공백기술 개발 및 수소 활용처 확대를 위한 사업을 지속적으로...,e
4,"2021년 기준 총 3,317명의 인원이 이수하여 및 위해성을 관리하고 있습니다. ...",e
...,...,...
3299,이와 같이 어떤 물체가 양전기와 음전기만을 띠는 대전체로부터 외부에 나타나는 전기적...,중립
3300,이 중 외부 시나리오에 따른 재무적 영향 측정이 필요한 요인은 8개인 것으로 확인되...,중립
3301,(87) 64 (94) 48 (92) 4 (100) 5 (71) 13 (57) 23...,중립
3302,전자결재 시스템과 연동된 전자계약 시스템을 운영하여 사전 품의를 거쳐 정해진 표준계...,중립


In [ ]:
data.loc[(data['category'] == "중립"), 'category'] = 0
data.loc[(data['category'] == "e"), 'category'] = 1
data.loc[(data['category'] == "s"), 'category'] = 2
data.loc[(data['category'] == "g"), 'category'] = 3

data_list = []
for q, label in zip(data['contents'], data['category'])  :
    data1 = []
    data1.append(q)
    data1.append(str(label))

    data_list.append(data1)

In [ ]:
print(data_list[0])
print(data_list[100])
print(data_list[250])
print(data_list[1000])
print(data_list[2500])
print(data_list[3300])

['이를 유해화학물질 취급시설을 지속적으로 개선하여 화관법 취급시설 최초 검사 결과 발생을 사전 예방하기 위해 다양한 노력을 기울이고 있습니다. 통해, 화학 업계의 새로운 패러다임 변화에 적극 대응함으로써 신뢰할 수 있는 ‘적합’ 통보받았으며, 화학물질 누출 등의 사고를 미연에 방지하고자 노력하였습니다.', '1']
['현대자동차는 나아가 해당 에코닐을 당사 차량 소재로 적용하는 방안까지 검토하고 있습니다. 업사이클링 프로젝트 ‘Re:Style’ 해양 생태계 복원 및 업사이클링 프로젝트 © Cor Kuyvenhoven 27 1.', '1']
['기후변화와 관련된 위험과 기회가 조직의 사업, 전략, 재무에 미치는 영향 c. 기후변화 시나리오를 고려한 조직 전략의 회복탄력성 C3.1 기후 변화로 발생하는 위험과 기회를 검토하고 있습니다. 기후변화 리스크를 중대성에 따라 위험단계별로 식별하고 있으며 단기, 중기, 장기로 단계별로 구분하여 관리하고 있습니다.', '1']
['또한 전문기관 컨설팅을 통해 사용계획서 등을 통해 작업 전 사전 승인을 득한 후, 안전관리자가 사업장 안전관리 상태 조사 및 기술 지도를 진행해 안전보건 개선 안전교육을 실시한 뒤에 작업이 진행될 수 있도록 하고 있습니다. 사항을 도출하고 이행하며 재해 발생을 예방하고 있습니다.', '2']
['품질경영회의 # 단순/중대 하자 판단 : 품질조직 팀장 회의 안건 상정여부 결정 지침개정 필요 시 회의 의사결정 필요 Data 분석/검토 즉시 조치 가능 Zero Defect 회의 상품/설계 표준 현장 적용 품질 이슈 발생 ·공사 중 품질이슈(주택 사업부 취합) ·준공 현장 하자(e편한세상품질팀, 아크로품질팀) 단순 오시공 주택사업부 중대하자 및 품질이슈 품질이슈 대책회의 해당 현장 전수조사 및 여건보고 하자인식 및 실적 모니터링 품질경영회의 지침 기준 수립/상품 적용 시공성 + 하자개선 여부 확인 # 본부장 참석/결정안건 보고 전사적 품질경영(TQM) 고객 Action Plan 고객의 소리에 기반한 고객

In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data, test_size=0.25, random_state=0)
print(len(dataset_train))
print(len(dataset_test))

2478
826


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([dataset.iloc[i][sent_idx]]) for i in range(len(dataset))]
        self.labels = [np.int32(dataset.iloc[i][label_idx]) for i in range(len(dataset))]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5, shuffle=True)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


# **KOBERT 학습시키기**

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=4,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device), return_dict=False)

        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out) 

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

Train

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()
        train_acc += calc_accuracy(out, label)
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

TEST

In [ ]:
def softmax(vals, idx):
    valscpu = vals.cpu().detach().squeeze(0)
    a = 0
    for i in valscpu:
        a += np.exp(i)
    return ((np.exp(valscpu[idx]))/a).item() * 100

def testModel(model, seq):
    cate = ["중립","e","s","g"]
    tmp = [seq]
    transform = nlp.data.BERTSentenceTransform(tok, max_len, pad=True, pair=False)
    tokenized = transform(tmp)

    model.eval()
    result = model(torch.tensor([tokenized[0]]).to(device), [tokenized[1]], torch.tensor(tokenized[2]).to(device))
    idx = result.argmax().cpu().item()
    print("보고서의 카테고리는:", cate[idx])
    print("신뢰도는:", "{:.2f}%".format(softmax(result,idx)))

In [ ]:
testModel(model, "이사회 금호석유화학은 지속가능한 기업을 만들기 위해 건전한 지배구조를 구축하고 있습니다. 이사회는 이해관계자의 이익을 대변하고, 경영진에 대한 감독 역할을 하며, 장기적인 관점의 의사결정을 하기 위해 노력합니다.")

보고서의 카테고리는: g
신뢰도는: 99.55%


<ipython-input-21-d24c5ecba32b>:15: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  result = model(torch.tensor([tokenized[0]]).to(device), [tokenized[1]], torch.tensor(tokenized[2]).to(device))


In [ ]:
testModel(model, "금호석유화학은 시장의 변화에 적절히 대응하고 친환경 포트폴리오 전환을 위해 고부가/친환경 제품 생산, 친환경 자동차 관련 솔루션, 바이오/친환경소재 및 고부가 스페셜티 제품 연구개발 등을 계획 중입니다.")

보고서의 카테고리는: e
신뢰도는: 99.57%


In [ ]:
testModel(model, "당사는 금융상품과 관련하여 신용위험, 유동성위험 및 시장위험에 노출되어 있습니다. 본 주석은 당사가 노출되어 있는 위의 위험에 대한 정보와 당사의 위험관리 목표,정책, 위험 평가 및 관리 절차, 그리고 자본관리에 대해 공시하고 있습니다. 추가적인계량적 정보에 대해서는 본 재무제표 전반에 걸쳐서 공시되어 있습니다.")

보고서의 카테고리는: e
신뢰도는: 63.13%


In [ ]:
testModel(model, "주관하는 ‘2021년 자발적에너지효율목표제 시범사업’ 협약을 통해 에너지 원단위 목표 개선을 위해 노력하고 있으며, 지역사회 및 에너지시민연대에서 주관하는 환경 관련 활동에 참여하며 기후변화 대응 중요성에 대한 공감과 소통을 실천하고 있습니다. ")

보고서의 카테고리는: e
신뢰도는: 99.59%


In [ ]:
testModel(model, "생물다양성 유지")

보고서의 카테고리는: e
신뢰도는: 95.55%


In [ ]:
testModel(model, "생물다양성 유지 및 지속가능성을 추진하는 국제 비영리 환경보호단체")

보고서의 카테고리는: e
신뢰도는: 99.53%


In [ ]:
testModel(model, "아울러 제품 제조, 판매 전단계에 있어서의 탄소배출절감을 위한 공급망 관리 체계를 보다 강화해 나아갈 것입니다.")

보고서의 카테고리는: e
신뢰도는: 99.53%


In [ ]:
testModel(model, "개발에서 유통까지, 원료부터 제품까지, 모든 단계를 아우르는 품질안전의 확보는 필수적입니다.")

보고서의 카테고리는: s
신뢰도는: 61.94%


In [ ]:
testModel(model, "롯데제과는 동반성장아카데미를 온라인으로 연중 운영하며 협력업체의 인적자원 개발을 지원하고 있습니다. ")

보고서의 카테고리는: 중립
신뢰도는: 99.42%


In [ ]:
testModel(model, "")

In [ ]:
print("testModel's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\w", model.state_dict()[param_tensor].size())

In [ ]:
type(model.state_dict())

collections.OrderedDict

In [ ]:
MODEL_PATH = "saved"
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)
torch.save(model.state_dict(), os.path.join(MODEL_PATH,"testModel.pt"))